In [17]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import Flowers102

# Define transforms for data normalization and augmentation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to match input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Standard normalization
])

# Load Flowers102 dataset
train_dataset = Flowers102(root='./data', split='test', transform=transform, download=True)
test_dataset = Flowers102(root='./data', split='train', transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [18]:
if torch.cuda.is_available():
    device = 'cuda:0'
    print("[INFO] Current device:", torch.cuda.get_device_name(torch.cuda.current_device()), f", device num:{torch.cuda.current_device()}")
elif torch.has_mps:
    device = 'mps'
    print("[INFO] Current device: MAC OS Metal Performance Shaders.")
else:
    device = 'cpu'
    print("[INFO] Current device: CPU")
    print("*"*50,"\n[WARNING] You may need to change the device to GPU or MPS to get better performance.")
    print("*"*50)
device = torch.device(device)

[INFO] Current device: NVIDIA GeForce RTX 3090 Ti , device num:0


In [19]:
import torch.nn as nn
import torch.nn.functional as F

class FlowerNet(nn.Module):
    def __init__(self):
        super(FlowerNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 102)  # 102 flower classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 256 * 28 * 28)  # Flatten for fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = FlowerNet().to(device)


In [20]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# Training loop
for epoch in range(50):  # Adjust epochs as necessary
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print(f'Epoch {epoch + 1}, Batch {i + 1}: loss = {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')


Epoch 1, Batch 100: loss = 4.486
Epoch 2, Batch 100: loss = 4.111
Epoch 3, Batch 100: loss = 3.426
Epoch 4, Batch 100: loss = 2.832
Epoch 5, Batch 100: loss = 2.454
Epoch 6, Batch 100: loss = 2.100
Epoch 7, Batch 100: loss = 1.820
Epoch 8, Batch 100: loss = 1.492
Epoch 9, Batch 100: loss = 1.165
Epoch 10, Batch 100: loss = 0.917
Epoch 11, Batch 100: loss = 0.684
Epoch 12, Batch 100: loss = 0.495
Epoch 13, Batch 100: loss = 0.273
Epoch 14, Batch 100: loss = 0.253
Epoch 15, Batch 100: loss = 0.145
Epoch 16, Batch 100: loss = 0.116
Epoch 17, Batch 100: loss = 0.056
Epoch 18, Batch 100: loss = 0.008
Epoch 19, Batch 100: loss = 0.004
Epoch 20, Batch 100: loss = 0.002
Epoch 21, Batch 100: loss = 0.008
Epoch 22, Batch 100: loss = 0.001
Epoch 23, Batch 100: loss = 0.001
Epoch 24, Batch 100: loss = 0.007
Epoch 25, Batch 100: loss = 0.006
Epoch 26, Batch 100: loss = 0.003
Epoch 27, Batch 100: loss = 0.001
Epoch 28, Batch 100: loss = 0.001
Epoch 29, Batch 100: loss = 0.001
Epoch 30, Batch 100: lo

In [21]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test images: {100 * correct / total}%')


Accuracy on test images: 40.490196078431374%
